Mapa z punktami pomiarowymi 

In [1]:
import sqlite3
import folium

def create_station_map(database_name: str):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    cursor.execute("SELECT station_name, latitude, longitude FROM STATIONS WHERE latitude IS NOT NULL AND longitude IS NOT NULL")
    stations = cursor.fetchall()
    conn.close()

    poland_center = [52.0, 19.0]

    mapa = folium.Map(location=poland_center, zoom_start=6)

    for name, lat, lon in stations:
        folium.Marker(
            location=[lat, lon],
            popup=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(mapa)

    return mapa

create_station_map("data.db")


Wyświetlenie wartości temperaturowych w punktach

In [2]:
import sqlite3
import folium

def create_station_map(database_name: str):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    # Pobranie najnowszej daty i godziny dla każdej stacji i dołączenie temperatury
    query = """
    SELECT s.station_name, s.latitude, s.longitude, m.Temperature
    FROM STATIONS s
    JOIN (
        SELECT Station_id, Temperature
        FROM METEO
        WHERE (Date, Hour) IN (
            SELECT Date, Hour
            FROM METEO
            WHERE Station_id = m.Station_id
            ORDER BY Date DESC, Hour DESC
            LIMIT 1
        )
    ) m ON s.Station_id = m.Station_id
    WHERE s.latitude IS NOT NULL AND s.longitude IS NOT NULL
    """

    # Alternatywna i bardziej przenośna wersja zapytania SQL (SQLite nie obsługuje dobrze podzapytań z aliasami w WHERE)
    query = """
    SELECT s.station_name, s.latitude, s.longitude, m.Temperature
    FROM STATIONS s
    JOIN (
        SELECT Station_id, Temperature
        FROM METEO
        WHERE (Station_id, Date, Hour) IN (
            SELECT Station_id, MAX(Date), MAX(Hour)
            FROM METEO
            GROUP BY Station_id
        )
    ) m ON s.Station_id = m.Station_id
    WHERE s.latitude IS NOT NULL AND s.longitude IS NOT NULL
    """

    cursor.execute(query)
    stations = cursor.fetchall()
    conn.close()

    poland_center = [52.0, 19.0]
    mapa = folium.Map(location=poland_center, zoom_start=6)

    for name, lat, lon, temp in stations:
        popup_text = f"{name}<br>Temperature: {temp} °C"
        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(mapa)

    return mapa

create_station_map("data.db")


Pokolorowanie punktów

In [3]:
import sqlite3
import folium
import numpy as np

def temperature_to_color(temp, min_temp, max_temp):
    norm = (temp - min_temp) / (max_temp - min_temp) if max_temp != min_temp else 0.5
    red = int(255 * (1 - norm))
    green = int(255 * norm)
    return f'#{red:02x}{green:02x}00'

def create_station_map(database_name: str):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    query = """
    SELECT s.station_name, s.latitude, s.longitude, m.Temperature
    FROM STATIONS s
    JOIN (
        SELECT Station_id, Temperature
        FROM METEO
        WHERE (Station_id, Date, Hour) IN (
            SELECT Station_id, MAX(Date), MAX(Hour)
            FROM METEO
            GROUP BY Station_id
        )
    ) m ON s.Station_id = m.Station_id
    WHERE s.latitude IS NOT NULL AND s.longitude IS NOT NULL
    """

    cursor.execute(query)
    stations = cursor.fetchall()
    conn.close()

    # Konwersja temperatur i usuwanie wartości None
    temp_data = [(name, lat, lon, float(temp)) for name, lat, lon, temp in stations if temp is not None]
    temperatures = [temp for _, _, _, temp in temp_data]

    # Obliczenie IQR i filtracja outlierów
    q1 = np.percentile(temperatures, 25)
    q3 = np.percentile(temperatures, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Filtrowanie danych bez outlierów
    filtered_data = [(name, lat, lon, temp) for name, lat, lon, temp in temp_data if lower_bound <= temp <= upper_bound]
    if not filtered_data:
        return None

    min_temp = min(temp for _, _, _, temp in filtered_data)
    max_temp = max(temp for _, _, _, temp in filtered_data)

    poland_center = [52.0, 19.0]
    mapa = folium.Map(location=poland_center, zoom_start=6)

    for name, lat, lon, temp in filtered_data:
        color = temperature_to_color(temp, min_temp, max_temp)
        popup_text = f"{name}<br>Temperature: {temp:.1f} °C"
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            popup=popup_text,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9
        ).add_to(mapa)

    return mapa

create_station_map("data.db")


Przejście na siatkę gridową

In [7]:
import folium
import pandas as pd
import sqlite3
from matplotlib import cm, colors

conn = sqlite3.connect('data.db')

df = pd.read_sql_query("SELECT Longitude, Latitude, Temperature FROM Grid_Data", conn)

conn.close()

if df.empty:
    print("Brak danych w tabeli Grid_Data")
else:
    norm = colors.Normalize(vmin=df['Temperature'].min(), vmax=df['Temperature'].max())
    cmap = cm.get_cmap('coolwarm')

    center_lat = df['Latitude'].mean()
    center_lon = df['Longitude'].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    for _, row in df.iterrows():
        color = colors.rgb2hex(cmap(norm(row['Temperature'])))
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"Temp: {row['Temperature']:.2f}°C"
        ).add_to(m)

m

C:\Users\filip\AppData\Local\Temp\ipykernel_4648\3151464068.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


Przycięcie do granic Polski manualnie

In [ ]:
import folium
import pandas as pd
import sqlite3
from matplotlib import cm, colors


POLAND_BOUNDS = {
    'min_latitude': 49.0,
    'max_latitude': 55.0,
    'min_longitude': 14.0,
    'max_longitude': 24.5
}

conn = sqlite3.connect('data.db')

df = pd.read_sql_query("SELECT Longitude, Latitude, Temperature FROM Grid_Data", conn)

conn.close()

if df.empty:
    print("Brak danych w tabeli Grid_Data")
else:
    df_poland = df[
        (df['Latitude'] >= POLAND_BOUNDS['min_latitude']) &
        (df['Latitude'] <= POLAND_BOUNDS['max_latitude']) &
        (df['Longitude'] >= POLAND_BOUNDS['min_longitude']) &
        (df['Longitude'] <= POLAND_BOUNDS['max_longitude'])
    ]

    if df_poland.empty:
        print("Brak danych w tabeli Grid_Data w granicach Polski.")
    else:
        norm = colors.Normalize(vmin=df_poland['Temperature'].min(), vmax=df_poland['Temperature'].max())
        cmap = cm.get_cmap('coolwarm')

        center_lat = 52.0 
        center_lon = 19.0 

        m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

        for _, row in df_poland.iterrows():
            color = colors.rgb2hex(cmap(norm(row['Temperature'])))
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=4,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                popup=f"Temp: {row['Temperature']:.2f}°C"
            ).add_to(m)

        display(m) 

C:\Users\filip\AppData\Local\Temp\ipykernel_4648\4174373671.py:37: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


Przycięcie do granic polski używając plik JSON

In [3]:
import folium
import pandas as pd
import sqlite3
from matplotlib import cm, colors
import json
from shapely.geometry import Point, shape 


conn = sqlite3.connect('data.db')

df = pd.read_sql_query("SELECT Longitude, Latitude, Temperature FROM Grid_Data", conn)

conn.close()

if df.empty:
    print("Brak danych w tabeli Grid_Data")
else:
    try:
        with open('poland.country.json', 'r', encoding='utf-8') as f:
            poland_geojson = json.load(f)

        if poland_geojson['type'] == 'FeatureCollection':
            poland_geometry = poland_geojson['features'][0]['geometry']
        elif poland_geojson['type'] == 'Feature':
            poland_geometry = poland_geojson['geometry']
        else: 
            poland_geometry = poland_geojson

        poland_polygon = shape(poland_geometry)
        print("Pomyślnie wczytano plik poland.json i utworzono obiekt Polygon.")

    except FileNotFoundError:
        print("Błąd: Plik 'poland.json' nie został znaleziony. Upewnij się, że znajduje się w tym samym katalogu.")
        exit() 
    except json.JSONDecodeError:
        print("Błąd: Nie można odczytać pliku 'poland.json'. Sprawdź, czy jest to poprawny plik JSON.")
        exit()
    except KeyError as e:
        print(f"Błąd w strukturze pliku poland.json: Brak klucza {e}. Sprawdź, czy plik zawiera poprawną geometrię.")
        exit()
    except Exception as e:
        print(f"Wystąpił nieoczekiwany błąd podczas ładowania pliku poland.json: {e}")
        exit()


    df_poland_list = []
    for index, row in df.iterrows():
        point = Point(row['Longitude'], row['Latitude'])
        if poland_polygon.contains(point):
            df_poland_list.append(row)
    
    df_poland = pd.DataFrame(df_poland_list)

    if df_poland.empty:
        print("Brak danych w tabeli Grid_Data w granicach Polski (zgodnie z poland.json).")
    else:
        norm = colors.Normalize(vmin=df_poland['Temperature'].min(), vmax=df_poland['Temperature'].max())
        cmap = cm.get_cmap('coolwarm')

        center_lat = 52.0 
        center_lon = 19.0 

        m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

        for _, row in df_poland.iterrows():
            color = colors.rgb2hex(cmap(norm(row['Temperature'])))
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=4,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                popup=f"Temp: {row['Temperature']:.2f}°C"
            ).add_to(m)

        folium.GeoJson(poland_geojson, name='Poland Borders').add_to(m)
        print("Granice Polski dodano do mapy.")

m

Pomyślnie wczytano plik poland.json i utworzono obiekt Polygon.
Granice Polski dodano do mapy.


C:\Users\filip\AppData\Local\Temp\ipykernel_11276\1755492300.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')
